# Census Set

In [1]:
import pandas as pd
import numpy as np
import pickle
from utils import *
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, log_loss
from sklearn.model_selection import StratifiedKFold, cross_validate
from slice_finder import SliceFinder
import warnings
warnings.filterwarnings('ignore')

## Viewing regions identified for the Census set

In [2]:
with open('./Slices/Census.p', 'rb') as file:
    data = pickle.load(file)
    
    data = sorted(data, key=lambda d: d.size, reverse=True)
    for d in data:
        print(f'Description of the region:  {d.filters}')
        print(f'Region instances: {d.data_idx}')
        print(f'Region size:    {d.size}')
        print(f'Efect size: {d.effect_size}')
        print(f'Log Loss:    {d.metric}')
        print('\n')

Description of the region:  {'Martial Status': [[2]]}
Region instances: Index([    1,     3,     4,     5,     7,     9,    10,    11,    15,    18,
       ...
       32533, 32545, 32547, 32550, 32551, 32552, 32554, 32556, 32557, 32560],
      dtype='int64', length=14065)
Region size:    14065
Efect size: 0.5798723269928144
Log Loss:    0.5539951272511505


Description of the region:  {'Relationship': [[0]]}
Region instances: Index([    1,     3,     7,     9,    10,    11,    15,    18,    20,    22,
       ...
       32522, 32526, 32532, 32533, 32547, 32550, 32551, 32552, 32554, 32557],
      dtype='int64', length=12463)
Region size:    12463
Efect size: 0.501722802648641
Log Loss:    0.540535012920946


Description of the region:  {'Relationship': [[5]]}
Region instances: Index([    4,     5,    37,    50,    52,    67,    82,   113,   125,   137,
       ...
       32421, 32425, 32428, 32479, 32485, 32513, 32528, 32545, 32556, 32560],
      dtype='int64', length=1406)
Region size:  

# Evaluating performance

In [4]:
adult_data_train = pd.read_csv(
    "data/adult.data",
    names=[
        "Age", "Workclass", "fnlwgt", "Education", "EducationNum", "MartialStatus",
        "Occupation", "Relationship", "Race", "Sex", "CapitalGain", "CapitalLoss",
        "Hoursperweek", "Country", "Target"],
        sep=r'\s*,\s*',
        engine='python',
        na_values="?")

# drop nan values
adult_data_train = adult_data_train.dropna()

# Encode categorical features
encoders = {}
for column in adult_data_train.columns:
    if adult_data_train.dtypes[column] == np.object_:
        le = LabelEncoder()
        adult_data_train[column] = le.fit_transform(adult_data_train[column])
        encoders[column] = le
        print(column, le.classes_, le.transform(le.classes_))

X_train, y_train = adult_data_train[adult_data_train.columns.difference(["Target"])], adult_data_train["Target"]

Workclass ['Federal-gov' 'Local-gov' 'Private' 'Self-emp-inc' 'Self-emp-not-inc'
 'State-gov' 'Without-pay'] [0 1 2 3 4 5 6]
Education ['10th' '11th' '12th' '1st-4th' '5th-6th' '7th-8th' '9th' 'Assoc-acdm'
 'Assoc-voc' 'Bachelors' 'Doctorate' 'HS-grad' 'Masters' 'Preschool'
 'Prof-school' 'Some-college'] [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
MartialStatus ['Divorced' 'Married-AF-spouse' 'Married-civ-spouse'
 'Married-spouse-absent' 'Never-married' 'Separated' 'Widowed'] [0 1 2 3 4 5 6]
Occupation ['Adm-clerical' 'Armed-Forces' 'Craft-repair' 'Exec-managerial'
 'Farming-fishing' 'Handlers-cleaners' 'Machine-op-inspct' 'Other-service'
 'Priv-house-serv' 'Prof-specialty' 'Protective-serv' 'Sales'
 'Tech-support' 'Transport-moving'] [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13]
Relationship ['Husband' 'Not-in-family' 'Other-relative' 'Own-child' 'Unmarried' 'Wife'] [0 1 2 3 4 5]
Race ['Amer-Indian-Eskimo' 'Asian-Pac-Islander' 'Black' 'Other' 'White'] [0 1 2 3 4]
Sex ['Female' 'Ma

In [3]:
adult_data_train.groupby('Target').describe()

Age                                                    Workclass  \
          count      mean        std   min   25%   50%   75%   max     count   
Target                                                                         
0       22654.0  36.60806  13.464631  17.0  26.0  34.0  45.0  90.0   22654.0   
1        7508.0  43.95911  10.269633  19.0  36.0  43.0  51.0  90.0    7508.0   

                  ... Hoursperweek        Country                            \
            mean  ...          75%   max    count       mean       std  min   
Target            ...                                                         
0       2.189415  ...         40.0  99.0  22654.0  36.300786  6.188487  0.0   
1       2.229222  ...         50.0  99.0   7508.0  36.629329  5.840897  0.0   

                                
         25%   50%   75%   max  
Target                          
0       38.0  38.0  38.0  40.0  
1       38.0  38.0  38.0  40.0  

[2 rows x 112 columns]

In [5]:
adult_data_test = pd.read_csv(
    "data/adult.data",
    names=[
        "Age", "Workclass", "fnlwgt", "Education", "EducationNum", "MartialStatus",
        "Occupation", "Relationship", "Race", "Sex", "CapitalGain", "CapitalLoss",
        "Hoursperweek", "Country", "Target"],
        sep=r'\s*,\s*',
        engine='python',
        na_values="?")

# drop nan values
adult_data_test = adult_data_test.dropna()

# Encode categorical features
encoders = {}
for column in adult_data_test.columns:
    if adult_data_test.dtypes[column] == np.object_:
        le = LabelEncoder()
        adult_data_test[column] = le.fit_transform(adult_data_test[column])
        encoders[column] = le
        print(column, le.classes_, le.transform(le.classes_))

X_val, y_val = adult_data_test[adult_data_test.columns.difference(["Target"])], adult_data_test["Target"]

Workclass ['Federal-gov' 'Local-gov' 'Private' 'Self-emp-inc' 'Self-emp-not-inc'
 'State-gov' 'Without-pay'] [0 1 2 3 4 5 6]
Education ['10th' '11th' '12th' '1st-4th' '5th-6th' '7th-8th' '9th' 'Assoc-acdm'
 'Assoc-voc' 'Bachelors' 'Doctorate' 'HS-grad' 'Masters' 'Preschool'
 'Prof-school' 'Some-college'] [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
MartialStatus ['Divorced' 'Married-AF-spouse' 'Married-civ-spouse'
 'Married-spouse-absent' 'Never-married' 'Separated' 'Widowed'] [0 1 2 3 4 5 6]
Occupation ['Adm-clerical' 'Armed-Forces' 'Craft-repair' 'Exec-managerial'
 'Farming-fishing' 'Handlers-cleaners' 'Machine-op-inspct' 'Other-service'
 'Priv-house-serv' 'Prof-specialty' 'Protective-serv' 'Sales'
 'Tech-support' 'Transport-moving'] [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13]
Relationship ['Husband' 'Not-in-family' 'Other-relative' 'Own-child' 'Unmarried' 'Wife'] [0 1 2 3 4 5]
Race ['Amer-Indian-Eskimo' 'Asian-Pac-Islander' 'Black' 'Other' 'White'] [0 1 2 3 4]
Sex ['Female' 'Ma

In [6]:
scoring = {
    'accuracy': 'accuracy',  
    'recall': 'recall',      
    'f1': 'f1',              
    'log_loss': make_scorer(log_loss, response_method='predict_proba')
}

start_time_model = time.time()
rf_optimized = model_train(X_train, y_train)
end_time_model = time.time()
execution_time_model = end_time_model - start_time_model

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
results_train = cross_validate(rf_optimized, X_train, y_train, cv=skf, scoring=scoring)

rf_optimized.fit(X_train, y_train)

y_val_pred = rf_optimized.predict(X_val)
y_val_proba = rf_optimized.predict_proba(X_val)

val_accuracy = np.mean(y_val_pred == y_val)
val_log_loss = log_loss(y_val, y_val_proba)


print("Time to train the model:", execution_time_model, "s")

print("\nMean and standard deviation of metrics in the training set (cross-validation):")
print(f"Accuracy: {np.mean(results_train['test_accuracy']):.2f} (+/- {np.std(results_train['test_accuracy']):.2f})")
print(f"Recall: {np.mean(results_train['test_recall']):.2f} (+/- {np.std(results_train['test_recall']):.2f})")
print(f"F1 Score: {np.mean(results_train['test_f1']):.2f} (+/- {np.std(results_train['test_f1']):.2f})")
print(f"Log-loss: {np.mean(results_train['test_log_loss']):.2f} (+/- {np.std(results_train['test_log_loss']):.2f})")

print("\nResults on the validation set:")
print(f"Accuracy: {val_accuracy:.2f}")
print(f"Log-loss: {val_log_loss:.2f}")
print("\nClassification report on the validation set:")
print(classification_report(y_val, y_val_pred))


Time to train the model: 549.8817100524902 s

Mean and standard deviation of metrics in the training set (cross-validation):
Accuracy: 0.86 (+/- 0.01)
Recall: 0.62 (+/- 0.02)
F1 Score: 0.69 (+/- 0.02)
Log-loss: 0.30 (+/- 0.01)

Results on the validation set:
Accuracy: 0.90
Log-loss: 0.23

Classification report on the validation set:
              precision    recall  f1-score   support

           0       0.91      0.96      0.93     22654
           1       0.86      0.70      0.77      7508

    accuracy                           0.90     30162
   macro avg       0.88      0.83      0.85     30162
weighted avg       0.89      0.90      0.89     30162



## Applying SMOTE to the entire set

In [7]:
scoring = {
    'accuracy': 'accuracy',  
    'recall': 'recall',      
    'f1': 'f1',              
    'log_loss': make_scorer(log_loss, response_method='predict_proba')
}

smote = SMOTE(sampling_strategy='auto', random_state=42)

y_slice = y_train
X_slice = X_train

X_slice_smote, y_slice_smote = smote.fit_resample(X_slice, y_slice)
region_SMOTE = (pd.concat([pd.DataFrame(X_slice_smote, columns=X_slice.columns), pd.Series(y_slice_smote, name='Target')], axis=1))

y_SMOTE_data = region_SMOTE['Target']
X_SMOTE_data = region_SMOTE.drop(columns='Target')

start_time_model = time.time()
rf_optimized = model_train(X_SMOTE_data, y_SMOTE_data)
end_time_model = time.time()
execution_time_model = end_time_model - start_time_model

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
results_train = cross_validate(rf_optimized, X_SMOTE_data, y_SMOTE_data, cv=skf, scoring=scoring)

rf_optimized.fit(X_SMOTE_data, y_SMOTE_data)

y_val_pred = rf_optimized.predict(X_val)
y_val_proba = rf_optimized.predict_proba(X_val)

val_accuracy = np.mean(y_val_pred == y_val)
val_log_loss = log_loss(y_val, y_val_proba)

print("Size of new dataset:", len(X_SMOTE_data))
print("Time to train the model:", execution_time_model, "s")

print("\nMean and standard deviation of metrics in the training set (cross-validation):")
print(f"Accuracy: {np.mean(results_train['test_accuracy']):.2f} (+/- {np.std(results_train['test_accuracy']):.2f})")
print(f"Recall: {np.mean(results_train['test_recall']):.2f} (+/- {np.std(results_train['test_recall']):.2f})")
print(f"F1 Score: {np.mean(results_train['test_f1']):.2f} (+/- {np.std(results_train['test_f1']):.2f})")
print(f"Log-loss: {np.mean(results_train['test_log_loss']):.2f} (+/- {np.std(results_train['test_log_loss']):.2f})")

print("\nResults on the validation set:")
print(f"Accuracy: {val_accuracy:.2f}")
print(f"Log-loss: {val_log_loss:.2f}")
print("\nClassification report on the validation set:")
print(classification_report(y_val, y_val_pred))

Size of new dataset: 45308
Time to train the model: 890.184894323349 s

Mean and standard deviation of metrics in the training set (cross-validation):
Accuracy: 0.89 (+/- 0.01)
Recall: 0.90 (+/- 0.01)
F1 Score: 0.89 (+/- 0.01)
Log-loss: 0.25 (+/- 0.01)

Results on the validation set:
Accuracy: 0.94
Log-loss: 0.16

Classification report on the validation set:
              precision    recall  f1-score   support

           0       0.97      0.95      0.96     22654
           1       0.85      0.91      0.88      7508

    accuracy                           0.94     30162
   macro avg       0.91      0.93      0.92     30162
weighted avg       0.94      0.94      0.94     30162



### Data augmentation guided by problem regions

In [7]:
region = adult_data_train.query('MartialStatus==2')
region.groupby('Target').describe()

Age                                                     Workclass  \
         count       mean        std   min   25%   50%   75%   max     count   
Target                                                                         
0       7666.0  41.636968  12.653064  17.0  32.0  40.0  50.0  90.0    7666.0   
1       6399.0  44.127989  10.112189  21.0  37.0  44.0  51.0  90.0    6399.0   

                  ... Hoursperweek       Country                            \
            mean  ...          75%   max   count       mean       std  min   
Target            ...                                                        
0       2.316332  ...         45.0  99.0  7666.0  36.148448  6.315194  0.0   
1       2.235506  ...         50.0  99.0  6399.0  36.614784  5.906077  0.0   

                                
         25%   50%   75%   max  
Target                          
0       38.0  38.0  38.0  40.0  
1       38.0  38.0  38.0  40.0  

[2 rows x 112 columns]

In [8]:
# Define evaluation metrics
scoring = {
    'accuracy': 'accuracy',  
    'recall': 'recall',      
    'f1': 'f1',              
    'log_loss': make_scorer(log_loss, response_method='predict_proba')
}

# Define the region
region = 'MartialStatus==2'

# Apply SMOTE in the region of interest
smote = SMOTE(sampling_strategy='auto', random_state=42)
X_region = X_train.query(region)  # Select only samples from the region
y_region = y_train[X_region.index]

# Apply SMOTE to the selected region
X_region_smote, y_region_smote = smote.fit_resample(X_region, y_region)

# Combine the original dataset with the balanced one to form the new training set
X_SMOTE_data = pd.concat([X_train.drop(X_region.index), pd.DataFrame(X_region_smote, columns=X_train.columns)])
y_SMOTE_data = pd.concat([y_train.drop(X_region.index), pd.Series(y_region_smote)])

# Train the model on the training set
start_time_model = time.time()
rf_optimized = model_train(X_SMOTE_data, y_SMOTE_data)
end_time_model = time.time()
execution_time_model = end_time_model - start_time_model

# Evaluation on the training set using cross-validation
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
results_train = cross_validate(rf_optimized, X_SMOTE_data, y_SMOTE_data, cv=skf, scoring=scoring)

# Train the model on the complete training data
rf_optimized.fit(X_SMOTE_data, y_SMOTE_data) 

# Evaluation on the validation set
y_val_pred = rf_optimized.predict(X_val)
y_val_proba = rf_optimized.predict_proba(X_val)

# Results on the validation set
val_accuracy = np.mean(y_val_pred == y_val)
val_log_loss = log_loss(y_val, y_val_proba)

# Display results
print("Time to train the model:", execution_time_model, "seconds")
print("Size of the new dataset:", len(X_SMOTE_data))
# Training results (cross-validation)
print("\nMean and standard deviation of metrics in the training set (cross-validation):")
print(f"Accuracy: {np.mean(results_train['test_accuracy']):.2f} (+/- {np.std(results_train['test_accuracy']):.2f})")
print(f"Recall: {np.mean(results_train['test_recall']):.2f} (+/- {np.std(results_train['test_recall']):.2f})")
print(f"F1 Score: {np.mean(results_train['test_f1']):.2f} (+/- {np.std(results_train['test_f1']):.2f})")
print(f"Log-loss: {np.mean(results_train['test_log_loss']):.2f} (+/- {np.std(results_train['test_log_loss']):.2f})")

# Results on the validation set
print("\nResults on the validation set:")
print(f"Accuracy: {val_accuracy:.2f}")
print(f"Log-loss: {val_log_loss:.2f}")
print("\nClassification report on the validation set:")
print(classification_report(y_val, y_val_pred))

Time to train the model: 649.7774453163147 seconds
Size of the new dataset: 31429

Mean and standard deviation of metrics in the training set (cross-validation):
Accuracy: 0.85 (+/- 0.01)
Recall: 0.67 (+/- 0.01)
F1 Score: 0.72 (+/- 0.01)
Log-loss: 0.32 (+/- 0.01)

Results on the validation set:
Accuracy: 0.88
Log-loss: 0.26

Classification report on the validation set:
              precision    recall  f1-score   support

           0       0.90      0.95      0.92     22654
           1       0.81      0.69      0.75      7508

    accuracy                           0.88     30162
   macro avg       0.86      0.82      0.84     30162
weighted avg       0.88      0.88      0.88     30162



In [9]:
region = adult_data_train.query('Relationship==0')
region.groupby('Target').describe()

Age                                                     Workclass  \
         count       mean        std   min   25%   50%   75%   max     count   
Target                                                                         
0       6784.0  42.131191  12.643627  17.0  32.0  40.0  51.0  90.0    6784.0   
1       5679.0  44.561719  10.074241  22.0  37.0  44.0  51.0  90.0    5679.0   

                  ... Hoursperweek       Country                            \
            mean  ...          75%   max   count       mean       std  min   
Target            ...                                                        
0       2.330041  ...         48.0  99.0  6784.0  36.283903  6.090564  0.0   
1       2.244409  ...         50.0  99.0  5679.0  36.684804  5.727642  0.0   

                                
         25%   50%   75%   max  
Target                          
0       38.0  38.0  38.0  40.0  
1       38.0  38.0  38.0  40.0  

[2 rows x 112 columns]

In [9]:
# Define evaluation metrics
scoring = {
    'accuracy': 'accuracy',  
    'recall': 'recall',      
    'f1': 'f1',              
    'log_loss': make_scorer(log_loss, response_method='predict_proba')
}

# Define the region
region = 'Relationship==0'

# Apply SMOTE in the region of interest
smote = SMOTE(sampling_strategy='auto', random_state=42)
X_region = X_train.query(region)  # Select only samples from the region
y_region = y_train[X_region.index]

# Apply SMOTE to the selected region
X_region_smote, y_region_smote = smote.fit_resample(X_region, y_region)

# Combine the original dataset with the balanced one to form the new training set
X_SMOTE_data = pd.concat([X_train.drop(X_region.index), pd.DataFrame(X_region_smote, columns=X_train.columns)])
y_SMOTE_data = pd.concat([y_train.drop(X_region.index), pd.Series(y_region_smote)])

# Train the model on the training set
start_time_model = time.time()
rf_optimized = model_train(X_SMOTE_data, y_SMOTE_data)
end_time_model = time.time()
execution_time_model = end_time_model - start_time_model

# Evaluation on the training set using cross-validation
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
results_train = cross_validate(rf_optimized, X_SMOTE_data, y_SMOTE_data, cv=skf, scoring=scoring)

# Train the model on the complete training data
rf_optimized.fit(X_SMOTE_data, y_SMOTE_data) 

# Evaluation on the validation set
y_val_pred = rf_optimized.predict(X_val)
y_val_proba = rf_optimized.predict_proba(X_val)

# Results on the validation set
val_accuracy = np.mean(y_val_pred == y_val)
val_log_loss = log_loss(y_val, y_val_proba)

# Display results
print("Time to train the model:", execution_time_model, "seconds")
print("Size of the new dataset:", len(X_SMOTE_data))
# Training results (cross-validation)
print("\nMean and standard deviation of metrics in the training set (cross-validation):")
print(f"Accuracy: {np.mean(results_train['test_accuracy']):.2f} (+/- {np.std(results_train['test_accuracy']):.2f})")
print(f"Recall: {np.mean(results_train['test_recall']):.2f} (+/- {np.std(results_train['test_recall']):.2f})")
print(f"F1 Score: {np.mean(results_train['test_f1']):.2f} (+/- {np.std(results_train['test_f1']):.2f})")
print(f"Log-loss: {np.mean(results_train['test_log_loss']):.2f} (+/- {np.std(results_train['test_log_loss']):.2f})")

# Results on the validation set
print("\nResults on the validation set:")
print(f"Accuracy: {val_accuracy:.2f}")
print(f"Log-loss: {val_log_loss:.2f}")
print("\nClassification report on the validation set:")
print(classification_report(y_val, y_val_pred))

Time to train the model: 631.5649538040161 seconds
Size of the new dataset: 31267

Mean and standard deviation of metrics in the training set (cross-validation):
Accuracy: 0.86 (+/- 0.01)
Recall: 0.68 (+/- 0.01)
F1 Score: 0.73 (+/- 0.01)
Log-loss: 0.30 (+/- 0.01)

Results on the validation set:
Accuracy: 0.91
Log-loss: 0.21

Classification report on the validation set:
              precision    recall  f1-score   support

           0       0.93      0.96      0.94     22654
           1       0.87      0.77      0.82      7508

    accuracy                           0.91     30162
   macro avg       0.90      0.87      0.88     30162
weighted avg       0.91      0.91      0.91     30162



In [12]:
region = adult_data_train.query('CapitalGain==0 and Relationship==5')
region.groupby('Target').describe()

Age                                                     Workclass  \
        count       mean        std   min   25%   50%   75%   max     count   
Target                                                                        
0       666.0  38.833333  11.622670  18.0  29.0  38.0  47.0  77.0     666.0   
1       564.0  40.432624   9.661816  21.0  33.0  40.0  47.0  90.0     564.0   

                  ... Hoursperweek       Country                            \
            mean  ...          75%   max   count       mean       std  min   
Target            ...                                                        
0       2.213213  ...         40.0  99.0   666.0  35.848348  6.708729  1.0   
1       2.154255  ...         40.0  80.0   564.0  35.987589  7.293019  1.0   

                                
         25%   50%   75%   max  
Target                          
0       38.0  38.0  38.0  39.0  
1       38.0  38.0  38.0  40.0  

[2 rows x 112 columns]

In [10]:
# Define evaluation metrics
scoring = {
    'accuracy': 'accuracy',  
    'recall': 'recall',      
    'f1': 'f1',              
    'log_loss': make_scorer(log_loss, response_method='predict_proba')
}

# Define the region
region = 'CapitalGain==0 and Relationship==5'

# Apply SMOTE in the region of interest
smote = SMOTE(sampling_strategy='auto', random_state=42)
X_region = X_train.query(region)  # Select only samples from the region
y_region = y_train[X_region.index]

# Apply SMOTE to the selected region
X_region_smote, y_region_smote = smote.fit_resample(X_region, y_region)

# Combine the original dataset with the balanced one to form the new training set
X_SMOTE_data = pd.concat([X_train.drop(X_region.index), pd.DataFrame(X_region_smote, columns=X_train.columns)])
y_SMOTE_data = pd.concat([y_train.drop(X_region.index), pd.Series(y_region_smote)])

# Train the model on the training set
start_time_model = time.time()
rf_optimized = model_train(X_SMOTE_data, y_SMOTE_data)
end_time_model = time.time()
execution_time_model = end_time_model - start_time_model

# Evaluation on the training set using cross-validation
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
results_train = cross_validate(rf_optimized, X_SMOTE_data, y_SMOTE_data, cv=skf, scoring=scoring)

# Train the model on the complete training data
rf_optimized.fit(X_SMOTE_data, y_SMOTE_data) 

# Evaluation on the validation set
y_val_pred = rf_optimized.predict(X_val)
y_val_proba = rf_optimized.predict_proba(X_val)

# Results on the validation set
val_accuracy = np.mean(y_val_pred == y_val)
val_log_loss = log_loss(y_val, y_val_proba)

# Display results
print("Time to train the model:", execution_time_model, "seconds")
print("Size of the new dataset:", len(X_SMOTE_data))
# Training results (cross-validation)
print("\nMean and standard deviation of metrics in the training set (cross-validation):")
print(f"Accuracy: {np.mean(results_train['test_accuracy']):.2f} (+/- {np.std(results_train['test_accuracy']):.2f})")
print(f"Recall: {np.mean(results_train['test_recall']):.2f} (+/- {np.std(results_train['test_recall']):.2f})")
print(f"F1 Score: {np.mean(results_train['test_f1']):.2f} (+/- {np.std(results_train['test_f1']):.2f})")
print(f"Log-loss: {np.mean(results_train['test_log_loss']):.2f} (+/- {np.std(results_train['test_log_loss']):.2f})")

# Results on the validation set
print("\nResults on the validation set:")
print(f"Accuracy: {val_accuracy:.2f}")
print(f"Log-loss: {val_log_loss:.2f}")
print("\nClassification report on the validation set:")
print(classification_report(y_val, y_val_pred))

Time to train the model: 472.51897382736206 seconds
Size of the new dataset: 30264

Mean and standard deviation of metrics in the training set (cross-validation):
Accuracy: 0.86 (+/- 0.01)
Recall: 0.64 (+/- 0.01)
F1 Score: 0.70 (+/- 0.01)
Log-loss: 0.31 (+/- 0.01)

Results on the validation set:
Accuracy: 0.93
Log-loss: 0.18

Classification report on the validation set:
              precision    recall  f1-score   support

           0       0.94      0.97      0.95     22654
           1       0.90      0.80      0.85      7508

    accuracy                           0.93     30162
   macro avg       0.92      0.89      0.90     30162
weighted avg       0.93      0.93      0.93     30162



In [14]:
region = adult_data_train.query('Education==14 and CapitalGain==0')
region.groupby('Target').describe()

Age                                                       Workclass  \
        count       mean        std   min    25%   50%    75%   max     count   
Target                                                                          
0       128.0  42.945312  15.263207  25.0  29.75  40.0  53.25  88.0     128.0   
1       275.0  44.040000   9.629365  26.0  37.00  43.0  50.00  80.0     275.0   

                  ... Hoursperweek       Country                            \
            mean  ...          75%   max   count       mean       std  min   
Target            ...                                                        
0       2.515625  ...         50.0  99.0   128.0  35.585938  7.725128  1.0   
1       2.654545  ...         55.0  99.0   275.0  36.021818  6.602330  3.0   

                                
         25%   50%   75%   max  
Target                          
0       38.0  38.0  38.0  38.0  
1       38.0  38.0  38.0  38.0  

[2 rows x 112 columns]

In [11]:
# Define evaluation metrics
scoring = {
    'accuracy': 'accuracy',  
    'recall': 'recall',      
    'f1': 'f1',              
    'log_loss': make_scorer(log_loss, response_method='predict_proba')
}

# Define the region
region = 'Education==14 and CapitalGain==0'

# Apply SMOTE in the region of interest
smote = SMOTE(sampling_strategy='auto', random_state=42)
X_region = X_train.query(region)  # Select only samples from the region
y_region = y_train[X_region.index]

# Apply SMOTE to the selected region
X_region_smote, y_region_smote = smote.fit_resample(X_region, y_region)

# Combine the original dataset with the balanced one to form the new training set
X_SMOTE_data = pd.concat([X_train.drop(X_region.index), pd.DataFrame(X_region_smote, columns=X_train.columns)])
y_SMOTE_data = pd.concat([y_train.drop(X_region.index), pd.Series(y_region_smote)])

# Train the model on the training set
start_time_model = time.time()
rf_optimized = model_train(X_SMOTE_data, y_SMOTE_data)
end_time_model = time.time()
execution_time_model = end_time_model - start_time_model

# Evaluation on the training set using cross-validation
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
results_train = cross_validate(rf_optimized, X_SMOTE_data, y_SMOTE_data, cv=skf, scoring=scoring)

# Train the model on the complete training data
rf_optimized.fit(X_SMOTE_data, y_SMOTE_data) 

# Evaluation on the validation set
y_val_pred = rf_optimized.predict(X_val)
y_val_proba = rf_optimized.predict_proba(X_val)

# Results on the validation set
val_accuracy = np.mean(y_val_pred == y_val)
val_log_loss = log_loss(y_val, y_val_proba)

# Display results
print("Time to train the model:", execution_time_model, "seconds")
print("Size of the new dataset:", len(X_SMOTE_data))
# Training results (cross-validation)
print("\nMean and standard deviation of metrics in the training set (cross-validation):")
print(f"Accuracy: {np.mean(results_train['test_accuracy']):.2f} (+/- {np.std(results_train['test_accuracy']):.2f})")
print(f"Recall: {np.mean(results_train['test_recall']):.2f} (+/- {np.std(results_train['test_recall']):.2f})")
print(f"F1 Score: {np.mean(results_train['test_f1']):.2f} (+/- {np.std(results_train['test_f1']):.2f})")
print(f"Log-loss: {np.mean(results_train['test_log_loss']):.2f} (+/- {np.std(results_train['test_log_loss']):.2f})")

# Results on the validation set
print("\nResults on the validation set:")
print(f"Accuracy: {val_accuracy:.2f}")
print(f"Log-loss: {val_log_loss:.2f}")
print("\nClassification report on the validation set:")
print(classification_report(y_val, y_val_pred))

Time to train the model: 603.9411723613739 seconds
Size of the new dataset: 30309

Mean and standard deviation of metrics in the training set (cross-validation):
Accuracy: 0.86 (+/- 0.00)
Recall: 0.61 (+/- 0.01)
F1 Score: 0.69 (+/- 0.01)
Log-loss: 0.30 (+/- 0.01)

Results on the validation set:
Accuracy: 0.89
Log-loss: 0.24

Classification report on the validation set:
              precision    recall  f1-score   support

           0       0.90      0.96      0.93     22654
           1       0.85      0.68      0.76      7508

    accuracy                           0.89     30162
   macro avg       0.88      0.82      0.84     30162
weighted avg       0.89      0.89      0.89     30162



In [16]:
region = adult_data_train.query('CapitalGain==0 and EducationNum==15')
region.groupby('Target').describe()

Age                                                       Workclass  \
        count       mean        std   min    25%   50%    75%   max     count   
Target                                                                          
0       128.0  42.945312  15.263207  25.0  29.75  40.0  53.25  88.0     128.0   
1       275.0  44.040000   9.629365  26.0  37.00  43.0  50.00  80.0     275.0   

                  ... Hoursperweek       Country                            \
            mean  ...          75%   max   count       mean       std  min   
Target            ...                                                        
0       2.515625  ...         50.0  99.0   128.0  35.585938  7.725128  1.0   
1       2.654545  ...         55.0  99.0   275.0  36.021818  6.602330  3.0   

                                
         25%   50%   75%   max  
Target                          
0       38.0  38.0  38.0  38.0  
1       38.0  38.0  38.0  38.0  

[2 rows x 112 columns]

In [12]:
# Define evaluation metrics
scoring = {
    'accuracy': 'accuracy',  
    'recall': 'recall',      
    'f1': 'f1',              
    'log_loss': make_scorer(log_loss, response_method='predict_proba')
}

# Define the region
region = 'CapitalGain==0 and EducationNum==15'

# Apply SMOTE in the region of interest
smote = SMOTE(sampling_strategy='auto', random_state=42)
X_region = X_train.query(region)  # Select only samples from the region
y_region = y_train[X_region.index]

# Apply SMOTE to the selected region
X_region_smote, y_region_smote = smote.fit_resample(X_region, y_region)

# Combine the original dataset with the balanced one to form the new training set
X_SMOTE_data = pd.concat([X_train.drop(X_region.index), pd.DataFrame(X_region_smote, columns=X_train.columns)])
y_SMOTE_data = pd.concat([y_train.drop(X_region.index), pd.Series(y_region_smote)])

# Train the model on the training set
start_time_model = time.time()
rf_optimized = model_train(X_SMOTE_data, y_SMOTE_data)
end_time_model = time.time()
execution_time_model = end_time_model - start_time_model

# Evaluation on the training set using cross-validation
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
results_train = cross_validate(rf_optimized, X_SMOTE_data, y_SMOTE_data, cv=skf, scoring=scoring)

# Train the model on the complete training data
rf_optimized.fit(X_SMOTE_data, y_SMOTE_data) 

# Evaluation on the validation set
y_val_pred = rf_optimized.predict(X_val)
y_val_proba = rf_optimized.predict_proba(X_val)

# Results on the validation set
val_accuracy = np.mean(y_val_pred == y_val)
val_log_loss = log_loss(y_val, y_val_proba)

# Display results
print("Time to train the model:", execution_time_model, "seconds")
print("Size of the new dataset:", len(X_SMOTE_data))
# Training results (cross-validation)
print("\nMean and standard deviation of metrics in the training set (cross-validation):")
print(f"Accuracy: {np.mean(results_train['test_accuracy']):.2f} (+/- {np.std(results_train['test_accuracy']):.2f})")
print(f"Recall: {np.mean(results_train['test_recall']):.2f} (+/- {np.std(results_train['test_recall']):.2f})")
print(f"F1 Score: {np.mean(results_train['test_f1']):.2f} (+/- {np.std(results_train['test_f1']):.2f})")
print(f"Log-loss: {np.mean(results_train['test_log_loss']):.2f} (+/- {np.std(results_train['test_log_loss']):.2f})")

# Results on the validation set
print("\nResults on the validation set:")
print(f"Accuracy: {val_accuracy:.2f}")
print(f"Log-loss: {val_log_loss:.2f}")
print("\nClassification report on the validation set:")
print(classification_report(y_val, y_val_pred))

Time to train the model: 637.5830273628235 seconds
Size of the new dataset: 30309

Mean and standard deviation of metrics in the training set (cross-validation):
Accuracy: 0.86 (+/- 0.00)
Recall: 0.61 (+/- 0.01)
F1 Score: 0.69 (+/- 0.01)
Log-loss: 0.30 (+/- 0.01)

Results on the validation set:
Accuracy: 0.89
Log-loss: 0.24

Classification report on the validation set:
              precision    recall  f1-score   support

           0       0.90      0.96      0.93     22654
           1       0.85      0.68      0.76      7508

    accuracy                           0.89     30162
   macro avg       0.88      0.82      0.84     30162
weighted avg       0.89      0.89      0.89     30162



### removal of the problematic region

In [15]:
def import_census():
    adult_data_train = pd.read_csv(
    "data/adult.data",
    names=[
        "Age", "Workclass", "fnlwgt", "Education", "EducationNum", "MartialStatus",
        "Occupation", "Relationship", "Race", "Sex", "CapitalGain", "CapitalLoss",
        "Hoursperweek", "Country", "Target"],
        sep=r'\s*,\s*',
        engine='python',
        na_values="?")

    # drop nan values
    adult_data_train = adult_data_train.dropna()

    # Encode categorical features
    encoders = {}
    for column in adult_data_train.columns:
        if adult_data_train.dtypes[column] == np.object_:
            le = LabelEncoder()
            adult_data_train[column] = le.fit_transform(adult_data_train[column])
            encoders[column] = le
            #print(column, le.classes_, le.transform(le.classes_))

    X_train, y_train = adult_data_train[adult_data_train.columns.difference(["Target"])], adult_data_train["Target"]
    
    return adult_data_train, X_train, y_train

In [16]:
adult_data_train_, X_train, y_train = import_census()

# Define evaluation metrics
scoring = {
    'accuracy': 'accuracy',  
    'recall': 'recall',      
    'f1': 'f1',              
    'log_loss': make_scorer(log_loss, response_method='predict_proba')
}

# Define the region for removal
region = 'CapitalGain==0 and Relationship==5'

# Identify and remove samples from the specified region
region_remove_idx = adult_data_train_.query(region).index
adult_data_train_.drop(region_remove_idx, errors='ignore', inplace=True)

# Ensure that the training set columns remain consistent
X_slice = adult_data_train_.drop(columns=['Target'])
y_slice = adult_data_train_['Target']

# Reindex the validation set to ensure consistency with the training set
X_val = X_val.reindex(columns=X_slice.columns, fill_value=0)

# Train the model on the training set
start_time_model = time.time()
rf_optimized = model_train(X_slice, y_slice)
end_time_model = time.time()
execution_time_model = end_time_model - start_time_model

# Evaluation on the training set using cross-validation
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
results_train = cross_validate(rf_optimized, X_slice, y_slice, cv=skf, scoring=scoring)

# Train the model on the complete training data
rf_optimized.fit(X_slice, y_slice)

# Evaluation on the validation set
y_val_pred = rf_optimized.predict(X_val)
y_val_proba = rf_optimized.predict_proba(X_val)

# Results on the validation set
val_accuracy = np.mean(y_val_pred == y_val)
val_log_loss = log_loss(y_val, y_val_proba)

# Display results
print("Time to train the model:", execution_time_model, "seconds")
print("Size of the dataset after removal:", len(X_slice))
# Training results (cross-validation)
print("\nMean and standard deviation of metrics in the training set (cross-validation):")
print(f"Accuracy: {np.mean(results_train['test_accuracy']):.2f} (+/- {np.std(results_train['test_accuracy']):.2f})")
print(f"Recall: {np.mean(results_train['test_recall']):.2f} (+/- {np.std(results_train['test_recall']):.2f})")
print(f"F1 Score: {np.mean(results_train['test_f1']):.2f} (+/- {np.std(results_train['test_f1']):.2f})")
print(f"Log-loss: {np.mean(results_train['test_log_loss']):.2f} (+/- {np.std(results_train['test_log_loss']):.2f})")

# Results on the validation set
print("\nResults on the validation set:")
print(f"Accuracy: {val_accuracy:.2f}")
print(f"Log-loss: {val_log_loss:.2f}")
print("\nClassification report on the validation set:")
print(classification_report(y_val, y_val_pred))

Time to train the model: 389.91152906417847 seconds
Size of the dataset after removal: 28932

Mean and standard deviation of metrics in the training set (cross-validation):
Accuracy: 0.87 (+/- 0.00)
Recall: 0.61 (+/- 0.02)
F1 Score: 0.69 (+/- 0.02)
Log-loss: 0.29 (+/- 0.00)

Results on the validation set:
Accuracy: 0.88
Log-loss: 0.26

Classification report on the validation set:
              precision    recall  f1-score   support

           0       0.88      0.96      0.92     22654
           1       0.84      0.62      0.71      7508

    accuracy                           0.88     30162
   macro avg       0.86      0.79      0.82     30162
weighted avg       0.87      0.88      0.87     30162



In [17]:
adult_data_train_, X_train, y_train = import_census()

# Define evaluation metrics
scoring = {
    'accuracy': 'accuracy',  
    'recall': 'recall',      
    'f1': 'f1',              
    'log_loss': make_scorer(log_loss, response_method='predict_proba')
}

# Define the region for removal
region = 'Education==14 and CapitalGain==0'

# Identify and remove samples from the specified region
region_remove_idx = adult_data_train_.query(region).index
adult_data_train_.drop(region_remove_idx, errors='ignore', inplace=True)

# Ensure that the training set columns remain consistent
X_slice = adult_data_train_.drop(columns=['Target'])
y_slice = adult_data_train_['Target']

# Reindex the validation set to ensure consistency with the training set
X_val = X_val.reindex(columns=X_slice.columns, fill_value=0)

# Train the model on the training set
start_time_model = time.time()
rf_optimized = model_train(X_slice, y_slice)
end_time_model = time.time()
execution_time_model = end_time_model - start_time_model

# Evaluation on the training set using cross-validation
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
results_train = cross_validate(rf_optimized, X_slice, y_slice, cv=skf, scoring=scoring)

# Train the model on the complete training data
rf_optimized.fit(X_slice, y_slice)

# Evaluation on the validation set
y_val_pred = rf_optimized.predict(X_val)
y_val_proba = rf_optimized.predict_proba(X_val)

# Results on the validation set
val_accuracy = np.mean(y_val_pred == y_val)
val_log_loss = log_loss(y_val, y_val_proba)

# Display results
print("Time to train the model:", execution_time_model, "seconds")
print("Size of the dataset after removal:", len(X_slice))
# Training results (cross-validation)
print("\nMean and standard deviation of metrics in the training set (cross-validation):")
print(f"Accuracy: {np.mean(results_train['test_accuracy']):.2f} (+/- {np.std(results_train['test_accuracy']):.2f})")
print(f"Recall: {np.mean(results_train['test_recall']):.2f} (+/- {np.std(results_train['test_recall']):.2f})")
print(f"F1 Score: {np.mean(results_train['test_f1']):.2f} (+/- {np.std(results_train['test_f1']):.2f})")
print(f"Log-loss: {np.mean(results_train['test_log_loss']):.2f} (+/- {np.std(results_train['test_log_loss']):.2f})")

# Results on the validation set
print("\nResults on the validation set:")
print(f"Accuracy: {val_accuracy:.2f}")
print(f"Log-loss: {val_log_loss:.2f}")
print("\nClassification report on the validation set:")
print(classification_report(y_val, y_val_pred))

Time to train the model: 647.5790123939514 seconds
Size of the dataset after removal: 29759

Mean and standard deviation of metrics in the training set (cross-validation):
Accuracy: 0.86 (+/- 0.01)
Recall: 0.61 (+/- 0.01)
F1 Score: 0.69 (+/- 0.01)
Log-loss: 0.30 (+/- 0.01)

Results on the validation set:
Accuracy: 0.92
Log-loss: 0.19

Classification report on the validation set:
              precision    recall  f1-score   support

           0       0.93      0.97      0.95     22654
           1       0.89      0.76      0.82      7508

    accuracy                           0.92     30162
   macro avg       0.91      0.87      0.89     30162
weighted avg       0.92      0.92      0.92     30162



In [18]:
adult_data_train_, X_train, y_train = import_census()

# Define evaluation metrics
scoring = {
    'accuracy': 'accuracy',  
    'recall': 'recall',      
    'f1': 'f1',              
    'log_loss': make_scorer(log_loss, response_method='predict_proba')
}

# Define the region for removal
region = 'CapitalGain==0 and EducationNum==15'

# Identify and remove samples from the specified region
region_remove_idx = adult_data_train_.query(region).index
adult_data_train_.drop(region_remove_idx, errors='ignore', inplace=True)

# Ensure that the training set columns remain consistent
X_slice = adult_data_train_.drop(columns=['Target'])
y_slice = adult_data_train_['Target']

# Reindex the validation set to ensure consistency with the training set
X_val = X_val.reindex(columns=X_slice.columns, fill_value=0)

# Train the model on the training set
start_time_model = time.time()
rf_optimized = model_train(X_slice, y_slice)
end_time_model = time.time()
execution_time_model = end_time_model - start_time_model

# Evaluation on the training set using cross-validation
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
results_train = cross_validate(rf_optimized, X_slice, y_slice, cv=skf, scoring=scoring)

# Train the model on the complete training data
rf_optimized.fit(X_slice, y_slice)

# Evaluation on the validation set
y_val_pred = rf_optimized.predict(X_val)
y_val_proba = rf_optimized.predict_proba(X_val)

# Results on the validation set
val_accuracy = np.mean(y_val_pred == y_val)
val_log_loss = log_loss(y_val, y_val_proba)

# Display results
print("Time to train the model:", execution_time_model, "seconds")
print("Size of the dataset after removal:", len(X_slice))
# Training results (cross-validation)
print("\nMean and standard deviation of metrics in the training set (cross-validation):")
print(f"Accuracy: {np.mean(results_train['test_accuracy']):.2f} (+/- {np.std(results_train['test_accuracy']):.2f})")
print(f"Recall: {np.mean(results_train['test_recall']):.2f} (+/- {np.std(results_train['test_recall']):.2f})")
print(f"F1 Score: {np.mean(results_train['test_f1']):.2f} (+/- {np.std(results_train['test_f1']):.2f})")
print(f"Log-loss: {np.mean(results_train['test_log_loss']):.2f} (+/- {np.std(results_train['test_log_loss']):.2f})")

# Results on the validation set
print("\nResults on the validation set:")
print(f"Accuracy: {val_accuracy:.2f}")
print(f"Log-loss: {val_log_loss:.2f}")
print("\nClassification report on the validation set:")
print(classification_report(y_val, y_val_pred))

Time to train the model: 601.7838935852051 seconds
Size of the dataset after removal: 29759

Mean and standard deviation of metrics in the training set (cross-validation):
Accuracy: 0.86 (+/- 0.01)
Recall: 0.61 (+/- 0.01)
F1 Score: 0.69 (+/- 0.01)
Log-loss: 0.30 (+/- 0.01)

Results on the validation set:
Accuracy: 0.92
Log-loss: 0.19

Classification report on the validation set:
              precision    recall  f1-score   support

           0       0.93      0.97      0.95     22654
           1       0.89      0.76      0.82      7508

    accuracy                           0.92     30162
   macro avg       0.91      0.87      0.89     30162
weighted avg       0.92      0.92      0.92     30162



In [19]:
adult_data_train_, X_train, y_train = import_census()

# Define evaluation metrics
scoring = {
    'accuracy': 'accuracy',  
    'recall': 'recall',      
    'f1': 'f1',              
    'log_loss': make_scorer(log_loss, response_method='predict_proba')
}

# Define the region for removal
region = 'Country==38 and Relationship==5'

# Identify and remove samples from the specified region
region_remove_idx = adult_data_train_.query(region).index
adult_data_train_.drop(region_remove_idx, errors='ignore', inplace=True)

# Ensure that the training set columns remain consistent
X_slice = adult_data_train_.drop(columns=['Target'])
y_slice = adult_data_train_['Target']

# Reindex the validation set to ensure consistency with the training set
X_val = X_val.reindex(columns=X_slice.columns, fill_value=0)

# Train the model on the training set
start_time_model = time.time()
rf_optimized = model_train(X_slice, y_slice)
end_time_model = time.time()
execution_time_model = end_time_model - start_time_model

# Evaluation on the training set using cross-validation
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
results_train = cross_validate(rf_optimized, X_slice, y_slice, cv=skf, scoring=scoring)

# Train the model on the complete training data
rf_optimized.fit(X_slice, y_slice)

# Evaluation on the validation set
y_val_pred = rf_optimized.predict(X_val)
y_val_proba = rf_optimized.predict_proba(X_val)

# Results on the validation set
val_accuracy = np.mean(y_val_pred == y_val)
val_log_loss = log_loss(y_val, y_val_proba)

# Display results
print("Time to train the model:", execution_time_model, "seconds")
print("Size of the dataset after removal:", len(X_slice))
# Training results (cross-validation)
print("\nMean and standard deviation of metrics in the training set (cross-validation):")
print(f"Accuracy: {np.mean(results_train['test_accuracy']):.2f} (+/- {np.std(results_train['test_accuracy']):.2f})")
print(f"Recall: {np.mean(results_train['test_recall']):.2f} (+/- {np.std(results_train['test_recall']):.2f})")
print(f"F1 Score: {np.mean(results_train['test_f1']):.2f} (+/- {np.std(results_train['test_f1']):.2f})")
print(f"Log-loss: {np.mean(results_train['test_log_loss']):.2f} (+/- {np.std(results_train['test_log_loss']):.2f})")

# Results on the validation set
print("\nResults on the validation set:")
print(f"Accuracy: {val_accuracy:.2f}")
print(f"Log-loss: {val_log_loss:.2f}")
print("\nClassification report on the validation set:")
print(classification_report(y_val, y_val_pred))

Time to train the model: 596.266844034195 seconds
Size of the dataset after removal: 28911

Mean and standard deviation of metrics in the training set (cross-validation):
Accuracy: 0.87 (+/- 0.01)
Recall: 0.61 (+/- 0.01)
F1 Score: 0.69 (+/- 0.01)
Log-loss: 0.29 (+/- 0.01)

Results on the validation set:
Accuracy: 0.90
Log-loss: 0.22

Classification report on the validation set:
              precision    recall  f1-score   support

           0       0.91      0.97      0.93     22654
           1       0.87      0.70      0.77      7508

    accuracy                           0.90     30162
   macro avg       0.89      0.83      0.85     30162
weighted avg       0.90      0.90      0.89     30162



In [20]:
adult_data_train_, X_train, y_train = import_census()

# Define evaluation metrics
scoring = {
    'accuracy': 'accuracy',  
    'recall': 'recall',      
    'f1': 'f1',              
    'log_loss': make_scorer(log_loss, response_method='predict_proba')
}

# Define the region for removal
region = 'Relationship==5 and Race==4'

# Identify and remove samples from the specified region
region_remove_idx = adult_data_train_.query(region).index
adult_data_train_.drop(region_remove_idx, errors='ignore', inplace=True)

# Ensure that the training set columns remain consistent
X_slice = adult_data_train_.drop(columns=['Target'])
y_slice = adult_data_train_['Target']

# Reindex the validation set to ensure consistency with the training set
X_val = X_val.reindex(columns=X_slice.columns, fill_value=0)

# Train the model on the training set
start_time_model = time.time()
rf_optimized = model_train(X_slice, y_slice)
end_time_model = time.time()
execution_time_model = end_time_model - start_time_model

# Evaluation on the training set using cross-validation
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
results_train = cross_validate(rf_optimized, X_slice, y_slice, cv=skf, scoring=scoring)

# Train the model on the complete training data
rf_optimized.fit(X_slice, y_slice)

# Evaluation on the validation set
y_val_pred = rf_optimized.predict(X_val)
y_val_proba = rf_optimized.predict_proba(X_val)

# Results on the validation set
val_accuracy = np.mean(y_val_pred == y_val)
val_log_loss = log_loss(y_val, y_val_proba)

# Display results
print("Time to train the model:", execution_time_model, "seconds")
print("Size of the dataset after removal:", len(X_slice))
# Training results (cross-validation)
print("\nMean and standard deviation of metrics in the training set (cross-validation):")
print(f"Accuracy: {np.mean(results_train['test_accuracy']):.2f} (+/- {np.std(results_train['test_accuracy']):.2f})")
print(f"Recall: {np.mean(results_train['test_recall']):.2f} (+/- {np.std(results_train['test_recall']):.2f})")
print(f"F1 Score: {np.mean(results_train['test_f1']):.2f} (+/- {np.std(results_train['test_f1']):.2f})")
print(f"Log-loss: {np.mean(results_train['test_log_loss']):.2f} (+/- {np.std(results_train['test_log_loss']):.2f})")

# Results on the validation set
print("\nResults on the validation set:")
print(f"Accuracy: {val_accuracy:.2f}")
print(f"Log-loss: {val_log_loss:.2f}")
print("\nClassification report on the validation set:")
print(classification_report(y_val, y_val_pred))

Time to train the model: 605.1188187599182 seconds
Size of the dataset after removal: 28973

Mean and standard deviation of metrics in the training set (cross-validation):
Accuracy: 0.87 (+/- 0.01)
Recall: 0.61 (+/- 0.02)
F1 Score: 0.69 (+/- 0.02)
Log-loss: 0.29 (+/- 0.01)

Results on the validation set:
Accuracy: 0.90
Log-loss: 0.22

Classification report on the validation set:
              precision    recall  f1-score   support

           0       0.91      0.97      0.94     22654
           1       0.89      0.70      0.78      7508

    accuracy                           0.90     30162
   macro avg       0.90      0.84      0.86     30162
weighted avg       0.90      0.90      0.90     30162



### random region removal

In [21]:
adult_data_train_, X_train, y_train = import_census()

# Define evaluation metrics
scoring = {
    'accuracy': 'accuracy',  
    'recall': 'recall',      
    'f1': 'f1',              
    'log_loss': make_scorer(log_loss, response_method='predict_proba')
}

# Randomly remove a defined number of rows
remove_rows = adult_data_train_.sample(n=len(adult_data_train_.query('Country==38 and Relationship==5').index)).index
adult_data_train_.drop(remove_rows, inplace=True)

# Prepare training data after removal
y_slice = adult_data_train_['Target']
X_slice = adult_data_train_.drop(columns=['Target'])

# Ensure validation set columns are aligned with the training set
X_val = X_val.reindex(columns=X_slice.columns, fill_value=0)

# Train the model on the training set
start_time_model = time.time()
rf_optimized = model_train(X_slice, y_slice)
end_time_model = time.time()
execution_time_model = end_time_model - start_time_model

# Evaluation on the training set using cross-validation
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
results_train = cross_validate(rf_optimized, X_slice, y_slice, cv=skf, scoring=scoring)

# Train the model on the complete training data
rf_optimized.fit(X_slice, y_slice)

# Evaluation on the validation set
y_val_pred = rf_optimized.predict(X_val)
y_val_proba = rf_optimized.predict_proba(X_val)

# Results on the validation set
val_accuracy = np.mean(y_val_pred == y_val)
val_log_loss = log_loss(y_val, y_val_proba)

# Display results
print("Time to train the model:", execution_time_model, "seconds")
print("Size of the dataset after removal:", len(X_slice))
# Training results (cross-validation)
print("\nMean and standard deviation of metrics in the training set (cross-validation):")
print(f"Accuracy: {np.mean(results_train['test_accuracy']):.2f} (+/- {np.std(results_train['test_accuracy']):.2f})")
print(f"Recall: {np.mean(results_train['test_recall']):.2f} (+/- {np.std(results_train['test_recall']):.2f})")
print(f"F1 Score: {np.mean(results_train['test_f1']):.2f} (+/- {np.std(results_train['test_f1']):.2f})")
print(f"Log-loss: {np.mean(results_train['test_log_loss']):.2f} (+/- {np.std(results_train['test_log_loss']):.2f})")

# Results on the validation set
print("\nResults on the validation set:")
print(f"Accuracy: {val_accuracy:.2f}")
print(f"Log-loss: {val_log_loss:.2f}")
print("\nClassification report on the validation set:")
print(classification_report(y_val, y_val_pred))

Time to train the model: 529.2392551898956 seconds
Size of the dataset after removal: 28911

Mean and standard deviation of metrics in the training set (cross-validation):
Accuracy: 0.86 (+/- 0.01)
Recall: 0.62 (+/- 0.02)
F1 Score: 0.69 (+/- 0.02)
Log-loss: 0.31 (+/- 0.01)

Results on the validation set:
Accuracy: 0.90
Log-loss: 0.23

Classification report on the validation set:
              precision    recall  f1-score   support

           0       0.91      0.96      0.93     22654
           1       0.86      0.71      0.78      7508

    accuracy                           0.90     30162
   macro avg       0.89      0.84      0.86     30162
weighted avg       0.90      0.90      0.90     30162



In [22]:
adult_data_train_, X_train, y_train = import_census()

# Define evaluation metrics
scoring = {
    'accuracy': 'accuracy',  
    'recall': 'recall',      
    'f1': 'f1',              
    'log_loss': make_scorer(log_loss, response_method='predict_proba')
}

# Randomly remove a defined number of rows
remove_rows = adult_data_train_.sample(n=len(adult_data_train_.query('CapitalGain==0 and EducationNum==15').index)).index
adult_data_train_.drop(remove_rows, inplace=True)

# Prepare training data after removal
y_slice = adult_data_train_['Target']
X_slice = adult_data_train_.drop(columns=['Target'])

# Ensure validation set columns are aligned with the training set
X_val = X_val.reindex(columns=X_slice.columns, fill_value=0)

# Train the model on the training set
start_time_model = time.time()
rf_optimized = model_train(X_slice, y_slice)
end_time_model = time.time()
execution_time_model = end_time_model - start_time_model

# Evaluation on the training set using cross-validation
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
results_train = cross_validate(rf_optimized, X_slice, y_slice, cv=skf, scoring=scoring)

# Train the model on the complete training data
rf_optimized.fit(X_slice, y_slice)

# Evaluation on the validation set
y_val_pred = rf_optimized.predict(X_val)
y_val_proba = rf_optimized.predict_proba(X_val)

# Results on the validation set
val_accuracy = np.mean(y_val_pred == y_val)
val_log_loss = log_loss(y_val, y_val_proba)

# Display results
print("Time to train the model:", execution_time_model, "seconds")
print("Size of the dataset after removal:", len(X_slice))
# Training results (cross-validation)
print("\nMean and standard deviation of metrics in the training set (cross-validation):")
print(f"Accuracy: {np.mean(results_train['test_accuracy']):.2f} (+/- {np.std(results_train['test_accuracy']):.2f})")
print(f"Recall: {np.mean(results_train['test_recall']):.2f} (+/- {np.std(results_train['test_recall']):.2f})")
print(f"F1 Score: {np.mean(results_train['test_f1']):.2f} (+/- {np.std(results_train['test_f1']):.2f})")
print(f"Log-loss: {np.mean(results_train['test_log_loss']):.2f} (+/- {np.std(results_train['test_log_loss']):.2f})")

# Results on the validation set
print("\nResults on the validation set:")
print(f"Accuracy: {val_accuracy:.2f}")
print(f"Log-loss: {val_log_loss:.2f}")
print("\nClassification report on the validation set:")
print(classification_report(y_val, y_val_pred))

Time to train the model: 513.727888584137 seconds
Size of the dataset after removal: 29759

Mean and standard deviation of metrics in the training set (cross-validation):
Accuracy: 0.86 (+/- 0.01)
Recall: 0.61 (+/- 0.02)
F1 Score: 0.69 (+/- 0.02)
Log-loss: 0.30 (+/- 0.01)

Results on the validation set:
Accuracy: 0.90
Log-loss: 0.24

Classification report on the validation set:
              precision    recall  f1-score   support

           0       0.90      0.96      0.93     22654
           1       0.86      0.69      0.77      7508

    accuracy                           0.90     30162
   macro avg       0.88      0.83      0.85     30162
weighted avg       0.89      0.90      0.89     30162



In [23]:
adult_data_train_, X_train, y_train = import_census()

# Define evaluation metrics
scoring = {
    'accuracy': 'accuracy',  
    'recall': 'recall',      
    'f1': 'f1',              
    'log_loss': make_scorer(log_loss, response_method='predict_proba')
}

# Randomly remove a defined number of rows
remove_rows = adult_data_train_.sample(n=len(adult_data_train_.query('CapitalGain==0 and Relationship==5').index)).index
adult_data_train_.drop(remove_rows, inplace=True)

# Prepare training data after removal
y_slice = adult_data_train_['Target']
X_slice = adult_data_train_.drop(columns=['Target'])

# Ensure validation set columns are aligned with the training set
X_val = X_val.reindex(columns=X_slice.columns, fill_value=0)

# Train the model on the training set
start_time_model = time.time()
rf_optimized = model_train(X_slice, y_slice)
end_time_model = time.time()
execution_time_model = end_time_model - start_time_model

# Evaluation on the training set using cross-validation
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
results_train = cross_validate(rf_optimized, X_slice, y_slice, cv=skf, scoring=scoring)

# Train the model on the complete training data
rf_optimized.fit(X_slice, y_slice)

# Evaluation on the validation set
y_val_pred = rf_optimized.predict(X_val)
y_val_proba = rf_optimized.predict_proba(X_val)

# Results on the validation set
val_accuracy = np.mean(y_val_pred == y_val)
val_log_loss = log_loss(y_val, y_val_proba)

# Display results
print("Time to train the model:", execution_time_model, "seconds")
print("Size of the dataset after removal:", len(X_slice))
# Training results (cross-validation)
print("\nMean and standard deviation of metrics in the training set (cross-validation):")
print(f"Accuracy: {np.mean(results_train['test_accuracy']):.2f} (+/- {np.std(results_train['test_accuracy']):.2f})")
print(f"Recall: {np.mean(results_train['test_recall']):.2f} (+/- {np.std(results_train['test_recall']):.2f})")
print(f"F1 Score: {np.mean(results_train['test_f1']):.2f} (+/- {np.std(results_train['test_f1']):.2f})")
print(f"Log-loss: {np.mean(results_train['test_log_loss']):.2f} (+/- {np.std(results_train['test_log_loss']):.2f})")

# Results on the validation set
print("\nResults on the validation set:")
print(f"Accuracy: {val_accuracy:.2f}")
print(f"Log-loss: {val_log_loss:.2f}")
print("\nClassification report on the validation set:")
print(classification_report(y_val, y_val_pred))

Time to train the model: 494.975501537323 seconds
Size of the dataset after removal: 28932

Mean and standard deviation of metrics in the training set (cross-validation):
Accuracy: 0.86 (+/- 0.00)
Recall: 0.61 (+/- 0.02)
F1 Score: 0.69 (+/- 0.01)
Log-loss: 0.30 (+/- 0.01)

Results on the validation set:
Accuracy: 0.89
Log-loss: 0.24

Classification report on the validation set:
              precision    recall  f1-score   support

           0       0.90      0.96      0.93     22654
           1       0.85      0.69      0.76      7508

    accuracy                           0.89     30162
   macro avg       0.88      0.82      0.85     30162
weighted avg       0.89      0.89      0.89     30162



In [24]:
adult_data_train_, X_train, y_train = import_census()

# Define evaluation metrics
scoring = {
    'accuracy': 'accuracy',  
    'recall': 'recall',      
    'f1': 'f1',              
    'log_loss': make_scorer(log_loss, response_method='predict_proba')
}

# Randomly remove a defined number of rows
remove_rows = adult_data_train_.sample(n=len(adult_data_train_.query('Relationship==5 and Race==4').index)).index
adult_data_train_.drop(remove_rows, inplace=True)

# Prepare training data after removal
y_slice = adult_data_train_['Target']
X_slice = adult_data_train_.drop(columns=['Target'])

# Ensure validation set columns are aligned with the training set
X_val = X_val.reindex(columns=X_slice.columns, fill_value=0)

# Train the model on the training set
start_time_model = time.time()
rf_optimized = model_train(X_slice, y_slice)
end_time_model = time.time()
execution_time_model = end_time_model - start_time_model

# Evaluation on the training set using cross-validation
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
results_train = cross_validate(rf_optimized, X_slice, y_slice, cv=skf, scoring=scoring)

# Train the model on the complete training data
rf_optimized.fit(X_slice, y_slice)

# Evaluation on the validation set
y_val_pred = rf_optimized.predict(X_val)
y_val_proba = rf_optimized.predict_proba(X_val)

# Results on the validation set
val_accuracy = np.mean(y_val_pred == y_val)
val_log_loss = log_loss(y_val, y_val_proba)

# Display results
print("Time to train the model:", execution_time_model, "seconds")
print("Size of the dataset after removal:", len(X_slice))
# Training results (cross-validation)
print("\nMean and standard deviation of metrics in the training set (cross-validation):")
print(f"Accuracy: {np.mean(results_train['test_accuracy']):.2f} (+/- {np.std(results_train['test_accuracy']):.2f})")
print(f"Recall: {np.mean(results_train['test_recall']):.2f} (+/- {np.std(results_train['test_recall']):.2f})")
print(f"F1 Score: {np.mean(results_train['test_f1']):.2f} (+/- {np.std(results_train['test_f1']):.2f})")
print(f"Log-loss: {np.mean(results_train['test_log_loss']):.2f} (+/- {np.std(results_train['test_log_loss']):.2f})")

# Results on the validation set
print("\nResults on the validation set:")
print(f"Accuracy: {val_accuracy:.2f}")
print(f"Log-loss: {val_log_loss:.2f}")
print("\nClassification report on the validation set:")
print(classification_report(y_val, y_val_pred))

Time to train the model: 562.6521492004395 seconds
Size of the dataset after removal: 28973

Mean and standard deviation of metrics in the training set (cross-validation):
Accuracy: 0.86 (+/- 0.01)
Recall: 0.61 (+/- 0.02)
F1 Score: 0.68 (+/- 0.01)
Log-loss: 0.30 (+/- 0.01)

Results on the validation set:
Accuracy: 0.88
Log-loss: 0.26

Classification report on the validation set:
              precision    recall  f1-score   support

           0       0.89      0.96      0.92     22654
           1       0.84      0.66      0.74      7508

    accuracy                           0.88     30162
   macro avg       0.87      0.81      0.83     30162
weighted avg       0.88      0.88      0.88     30162

